# Agriculture-Vision-task

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/Vison-for-Agriculture-project/vision-for-agriculture.zip' '/content' # move to content then unzip
%cd '/content'
!unzip 'vision-for-agriculture.zip'

In [1]:
import sys
import os
sys.path.append('/content/Agriculture-Vision')
os.chdir('/content/drive/MyDrive/Vison-for-Agriculture-project/Term_project_py')

In [2]:
import numpy as np
import torch
from torch import nn
from torch import functional as F
from torch import optim
from torch.utils.data import dataset, DataLoader, sampler, Subset
import matplotlib.pyplot as plt
from data_utils import *
from layer_utils import *
from Solver import *
import time

%load_ext autoreload
%autoreload 2
plt.rcParams['figure.figsize'] = (12.0, 8.0)

In [3]:
dtype = torch.float64
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device('cpu')
print(device)
print(torch.cuda.get_device_properties(0))

cuda
_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [4]:
train_path = '//content//Agriculture-Vision//train'
val_path = '//content//Agriculture-Vision//val'
train_dataset = Dataset_agriculture(train_path)
val_dataset = Dataset_agriculture(val_path)
small_loader = DataLoader(train_dataset, batch_size=4, sampler=sampler.SubsetRandomSampler(range(5000, 5005)), num_workers=4)
small_val_loader_abab = DataLoader(Subset(train_dataset, range(5000, 5008)), batch_size=4)

val_loader = DataLoader(val_dataset, batch_size=2, sampler=sampler.RandomSampler(range(0, len(val_dataset)), num_samples=20, replacement=True), num_workers=4)
full_loader = DataLoader(train_dataset, batch_size=2, num_workers=4)

In [ ]:
tic = time.time()
means, stds = compute_(small_loader)
toc = time.time()
print(toc-tic)

# tic = time.time()
# aaaa = []
# for all_data in small_loader:
#     train_data, _, _ = concencate_data(all_data)
#     aaaa.append(train_data)
# aaaa = torch.cat(aaaa, dim=0)
# test_mean = np.mean(aaaa.numpy(), axis=(0, 1, 2))
# test_std = np.std(aaaa.numpy(), axis=(0, 1, 2))
# toc = time.time()
# print(toc-tic)

means = tensor([[[[116.1301, 118.6523, 121.8633, 108.1013]]]])
stds = tensor([[[[45.3765, 42.2304, 41.7526, 44.9108]]]])

In [6]:
class U_Net(nn.Module):
    def __init__(self, input_channel, output_classes, bilinear=False):
        super().__init__() # when use nn.Module to build new module，must use nn.Module.__init__()
        factor = 2 if bilinear else 1
        self.doubleconv = DoubleConv(input_channel, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024 // factor)

        self.up4 = Up(1024, 512 // factor, bilinear)
        self.up3 = Up(512, 256 // factor, bilinear)
        self.up2 = Up(256, 128 // factor, bilinear)
        self.up1 = Up(128, 64, bilinear)
        self.output = OutConv(64, output_classes)

    def forward(self, X):
        X1 = self.doubleconv(X)
        X2 = self.down1(X1)
        X3 = self.down2(X2)
        X4 = self.down3(X3)
        X = self.down4(X4)

        X = self.up4(X, X4)
        X = self.up3(X, X3)
        X = self.up2(X, X2)
        X = self.up1(X, X1)
        out = self.output(X)
        return out

class Double_Unet(nn.Module):
  def __init__(self, input_channel, medium_channel, output_classes):
    super().__init__()
    self.first_U = U_Net(input_channel, medium_channel, bilinear=True)
    self.second_U = U_Net(medium_channel, output_classes)
  
  def forward(self, X):
    feature_pic = self.first_U(X)
    out = self.second_U(feature_pic)
    scores = torch.transpose(out, 1, 3)
    return scores

In [ ]:
learning_rate = 2e-4
model = Double_Unet(4, 4, 7)
model = model.to(device=device, dtype=dtype)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True) #训练两次再weight_decay
solver = Solver(model, optimizer, dataloader=full_loader, val_loader=val_loader, batch_time=50, epochs=1, print_every=100)
solver.train()

In [ ]:
classes = ['ground', 'cloud', 'doubleplant', 'planter_skip', 'strand_water', 'water_way', 'weed_cluster']
plt.figure(0, figsize=(10, 10))
plt.subplot(2,1,1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.grid()

plt.subplot(2, 1, 2)
abab = solver.val_acc
train_acc = np.vstack(solver.val_acc)
plt.plot(train_acc, '-o')
plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.legend(classes, loc='best')
plt.grid()

In [ ]:
load_path = '/content/drive/MyDrive/Vison-for-Agriculture-project/Term_project_py/model_8_10.pth'
package = torch.load(load_path)
solver = Solver(model, optimizer, dataloader=full_loader, val_loader=val_loader, batch_time=50, epochs=1, print_every=100)
solver._load(package)
# Continue training Part
learning_rate = 5e-4
optimizer = optim.Adam(solver.model.parameters(), lr=learning_rate)
solver.optimizer = optimizer
solver.train()

In [ ]:
#Inference
for pic in small_loader:
  picdate, masks = concencate_data(pic, means, stds, preprocess=True, test=True)
  plot_test_data = picdate * stds + means
  test_pic = picdate * masks
  preds = solver.predition(test_pic)
  plot_picture_contour(plot_test_data, masks, preds)

#Save

In [ ]:
save_path = '/content/drive/MyDrive/Vison-for-Agriculture-project/Term_project_py/model_8_9.pth'
torch.save(solver, save_path)

In [ ]:
load_path = '/content/drive/MyDrive/Vison-for-Agriculture-project/Term_project_py/model_8_8.pth'
package = torch.load(load_path)

# Overfit

In [ ]:
learning_rate = [1e-3, 5e-3, 1e-2, 5e-2]
optimize = ['SGD', 'Adam']
weights = [torch.tensor([1e-3, 1, 1, 1, 1, 1, 1], device=device), torch.tensor([1e-4, 1, 1, 1, 1, 1, 1], device=device)]
results = {}
for lr in learning_rate:
  for optimize_str in optimize:
    for weight in weights:
      print('Now is:', lr, optimize_str, weight)
      model = U_Net(4, 7)
      model = model.to(device=device, dtype=dtype)
      if optimize_str == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)
      else: optmizer = optim.Adam(model.parameters(), lr=lr)
      solver = Solver(model, optimizer, dataloader=small_val_loader_abab, val_loader=small_val_loader_abab, batch_time=2, 
                      epochs=10, print_every=2)
      solver.train()
      results[(lr, optimize_str, weight)] = (solver.loss_history, solver.val_acc)

In [ ]:
learning_rate = 5e-3
model = Double_Unet(4, 7, 3)
model = model.to(device=device, dtype=dtype)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True) #训练两次再weight_decay
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
solver = Solver(model, optimizer, dataloader=small_val_loader_abab, val_loader=small_val_loader_abab, loss_name='BCEDICE', batch_time=1, epochs=30, print_every=1)
solver.train()